# PEP Benchmarking - Guardian Dataset Both Algorithms

This notebook executes the PEP Benchmarking challenges on the Guardian Dataset for PEP Extraction Pipelines using automated algorithms for both Q-wave onset and B-point extraction.

## Setup and Helper Functions

In [1]:
import re
import json
from pprint import pprint
from pathlib import Path
from itertools import product

import pandas as pd
import numpy as np

from fau_colors import cmaps, register_fausans_font
from tqdm.auto import tqdm

import biopsykit as bp

import matplotlib.pyplot as plt
import seaborn as sns

from pepbench.algorithms.heartbeat_segmentation import HeartbeatSegmentationNeurokit
from pepbench.algorithms.ecg import QPeakExtractionNeurokitDwt, QWaveOnsetExtractionVanLien2013
from pepbench.algorithms.icg import BPointExtractionArbol2017, BPointExtractionDebski1993, BPointExtractionDrost2022, BPointExtractionForouzanfar2018, CPointExtractionScipyFindPeaks
from pepbench.algorithms.outlier_correction import OutlierCorrectionForouzanfar2018, OutlierCorrectionInterpolation, OutlierCorrectionDummy

from pepbench.datasets import GuardianDataset
from pepbench.pipelines import PepExtractionPipeline
from pepbench.evaluation import PepEvaluationChallenge

from IPython.display import Markdown


%matplotlib widget
%load_ext autoreload
%autoreload 2

In [2]:
#register_fausans_font()
plt.close("all")

palette = sns.color_palette(cmaps.faculties)
sns.set_theme(context="notebook", style="ticks", font="sans-serif", palette=palette)

plt.rcParams["figure.figsize"] = (10, 5)
plt.rcParams["pdf.fonttype"] = 42
plt.rcParams["mathtext.default"] = "regular"
plt.rcParams["font.family"] = "sans-serif"
#plt.rcParams["font.sans-serif"] = "FAUSans Office"

palette

[(0.0, 0.1843137254901961, 0.4235294117647059),
 (0.4666666666666667, 0.6235294117647059, 0.7098039215686275),
 (1.0, 0.7215686274509804, 0.10980392156862745),
 (0.0, 0.6392156862745098, 0.8784313725490196),
 (0.2627450980392157, 0.6901960784313725, 0.16470588235294117),
 (0.7843137254901961, 0.06274509803921569, 0.1803921568627451)]

In [3]:
root_path = Path("../../")

In [4]:
deploy_type = "local"

config_dict = json.load(root_path.joinpath("config.json").open(encoding="utf-8"))

guardian_base_path = Path(config_dict[deploy_type]["guardian_path"])
print(guardian_base_path)

\Users\sebas\Development\ResearchInternship\Data\2024_08_PEP_Benchmarking\Guardian_Dataset


In [5]:
result_path = root_path.joinpath("results/guardian_dataset_both_algorithms")
result_path.mkdir(exist_ok=True, parents=True)
result_path.resolve()

WindowsPath('C:/Users/sebas/Development/ResearchInternship/Code/pepbench/experiments/pep_algorithm_benchmarking/results/guardian_dataset_both_algorithms')

In [6]:
dataset_guardian = GuardianDataset(guardian_base_path, use_cache=True, only_labeled=True)
dataset_guardian

,participant,phase
0,GDN0005,Pause
1,GDN0005,Valsalva
2,GDN0005,HoldingBreath
3,GDN0005,TiltUp
4,GDN0005,TiltDown
...,...,...
101,GDN0029,TiltUp
102,GDN0029,TiltDown
103,GDN0030,Valsalva
104,GDN0030,HoldingBreath


## Initialize Algorithms

### Heartbeat Segmentation

In [7]:
heartbeat_segmentation_algo = HeartbeatSegmentationNeurokit()

### ECG - Q-Wave Onset

In [8]:
q_wave_algo_dict = {
    "dwt-neurokit": QPeakExtractionNeurokitDwt()
}
q_wave_algo_dict.update(**{f"r-peak-diff-{i}-ms": QWaveOnsetExtractionVanLien2013(time_interval_ms=i) for i in np.arange(32, 44, 2)})
q_wave_algos = list(q_wave_algo_dict.items())

print("Available Q-wave Onset algorithms:")
pprint(q_wave_algo_dict)

Available Q-wave Onset algorithms:
{'dwt-neurokit': QPeakExtractionNeurokitDwt(handle_missing_events='warn'),
 'r-peak-diff-32-ms': QWaveOnsetExtractionVanLien2013(time_interval_ms=32),
 'r-peak-diff-34-ms': QWaveOnsetExtractionVanLien2013(time_interval_ms=34),
 'r-peak-diff-36-ms': QWaveOnsetExtractionVanLien2013(time_interval_ms=36),
 'r-peak-diff-38-ms': QWaveOnsetExtractionVanLien2013(time_interval_ms=38),
 'r-peak-diff-40-ms': QWaveOnsetExtractionVanLien2013(time_interval_ms=40),
 'r-peak-diff-42-ms': QWaveOnsetExtractionVanLien2013(time_interval_ms=42)}


### ICG - B-Point

In [9]:
b_point_algo_dict = {
    "second-derivative": BPointExtractionDebski1993(),
    "third-derivative": BPointExtractionArbol2017(),
    "straight-line": BPointExtractionDrost2022(),
    "multiple-conditions": BPointExtractionForouzanfar2018()
}
b_point_algos = list(b_point_algo_dict.items())

print("Available B-point algorithms:")
pprint(b_point_algo_dict)

Available B-point algorithms:
{'multiple-conditions': BPointExtractionForouzanfar2018(correct_outliers=False, handle_missing_events='warn'),
 'second-derivative': BPointExtractionDebski1993(correct_outliers=False, handle_missing_events='warn'),
 'straight-line': BPointExtractionDrost2022(correct_outliers=False, handle_missing_events='warn'),
 'third-derivative': BPointExtractionArbol2017(handle_missing_events='warn', save_icg_derivatives=False, window_b_detection_ms=150)}


### Outlier Correction

In [10]:
outlier_correction_dict = {
    "none": OutlierCorrectionDummy(),
    "linear-interpolation": OutlierCorrectionInterpolation(),
    "autoregression": OutlierCorrectionForouzanfar2018()
}
outlier_correction_algos = list(outlier_correction_dict.items())

print("Available Outlier Correction algorithms:")
pprint(outlier_correction_dict)

Available Outlier Correction algorithms:
{'autoregression': OutlierCorrectionForouzanfar2018(),
 'linear-interpolation': OutlierCorrectionInterpolation(),
 'none': OutlierCorrectionDummy()}


In [11]:
algo_combinations = list(product(q_wave_algos, b_point_algos, outlier_correction_algos))

In [12]:
big_result_dict = {}

for algo_combi in tqdm(algo_combinations):
    algo_combi_names = [key[0] for key in algo_combi]

    filename_stub = "pep_results_guardian_" + "_".join(algo_combi_names)
    
    # check if exists
    if len(list(result_path.glob(f"{filename_stub}*"))) != 0:
        display(Markdown(f"Algrotihm combination {algo_combi_names} already challenged. Skipping..."))
        continue
    
    display(Markdown(f"Running algorithm combination {algo_combi_names}"))
    pep_challenge = PepEvaluationChallenge(dataset=dataset_guardian, validate_kwargs={"n_jobs": -1})

    pipeline = PepExtractionPipeline(
        heartbeat_segmentation_algo=heartbeat_segmentation_algo,
        q_wave_algo=algo_combi[0][1],
        b_point_algo=algo_combi[1][1],
        outlier_correction_algo=algo_combi[2][1],
        handle_negative_pep="nan",
        handle_missing_events="ignore"
    )
    pep_challenge.run(pipeline)
    pep_challenge.results_as_df()
    pep_challenge.save_results(result_path, filename_stub)

    big_result_dict[tuple(algo_combi_names)] = pep_challenge

  0%|          | 0/84 [00:00<?, ?it/s]

Algrotihm combination ['dwt-neurokit', 'second-derivative', 'none'] already challenged. Skipping...

Algrotihm combination ['dwt-neurokit', 'second-derivative', 'linear-interpolation'] already challenged. Skipping...

Algrotihm combination ['dwt-neurokit', 'second-derivative', 'autoregression'] already challenged. Skipping...

Algrotihm combination ['dwt-neurokit', 'third-derivative', 'none'] already challenged. Skipping...

...

Algrotihm combination ['r-peak-diff-42-ms', 'straight-line', 'linear-interpolation'] already challenged. Skipping...

Algrotihm combination ['r-peak-diff-42-ms', 'straight-line', 'autoregression'] already challenged. Skipping...

Algrotihm combination ['r-peak-diff-42-ms', 'multiple-conditions', 'none'] already challenged. Skipping...

Algrotihm combination ['r-peak-diff-42-ms', 'multiple-conditions', 'linear-interpolation'] already challenged. Skipping...

Algrotihm combination ['r-peak-diff-42-ms', 'multiple-conditions', 'autoregression'] already challenged. Skipping...